In [1]:
def h(map, location):
    """ Heuristic function for A* """
    return abs(location[0] - map.end[0]) + abs(location[1] - map.end[1])

In [9]:
class Map:
    def __init__(self, map):
        # self.map = map
        self.rows = len(map)
        self.cols = len(map[0])
        self.start = (0, 0)
        self.end = (self.rows - 1, self.cols - 1)
        self.passable = [[True if map[row][col] == 0 else False for col in range(self.cols)] for row in range(self.rows)]
        from collections import defaultdict
        self.g_score = defaultdict(lambda: float("inf"))
        self.g_score[self.start] = 1

        self.f_score = defaultdict(lambda: float("inf"))
        self.f_score[self.start] = h(self, self.start)

    def locations(self):
        """ Iterates over the locations in the map"""
        for row in range(self.rows):
            for col in range(self.cols):
                yield (row, col)

    def modify(self, location, value):
        self.passable[location[0]][location[1]] = value
        cost_of_changed = self.g_score[location]
        self.g_score[location] = float("inf")
        for loc in self.locations():
            if self.g_score[loc] >= cost_of_changed:
                self.g_score[loc] = float("inf")
                self.f_score[loc] = float("inf")

    def get_neighbours(self, loc):
        """ Return a list of neighbors for a given location """
        neighbours = []
        if loc[0] > 0:
            neighbours.append((loc[0] - 1, loc[1]))
        if loc[0] < self.rows - 1:
            neighbours.append((loc[0] + 1, loc[1]))
        if loc[1] > 0:
            neighbours.append((loc[0], loc[1] - 1))
        if loc[1] < self.cols - 1:
            neighbours.append((loc[0], loc[1] + 1))
        return neighbours

    def get_passable_neighbours(self, location):
        neigh = self.get_neighbours(location)
        return filter( lambda loc: self.passable[loc[0]][loc[1]], neigh)
        # def my_filter(neighbours):
        #     for loc in neighbours:
        #         if self.passable[loc[0]][loc[1]]:
        #             yield loc
        # return my_filter(neigh)

    def A_star(self):
        open_set = set([self.start])
        came_from = {}

        while open_set:
            current = min(open_set, key=lambda x: self.f_score[x])
            if current == self.end:
                # return reconstruct_path(came_from, current)
                return self.f_score[current]
            open_set.remove(current)
            for neighbor_loc in self.get_passable_neighbours(current):
                tentative_g_score = self.g_score[current] + 1
                if tentative_g_score < self.g_score[neighbor_loc]:
                    came_from[neighbor_loc] = current
                    self.g_score[neighbor_loc] = tentative_g_score
                    self.f_score[neighbor_loc] = self.g_score[neighbor_loc] + h(self, neighbor_loc)
                    if neighbor_loc not in open_set:
                        open_set.add(neighbor_loc)
        # No path found
        return float("inf")

In [10]:
def test(map):
    m = Map(map)
    print(m.A_star())
print("Para [[0]]")
test([[0]])
print("Para 2")
test([[0, 0], [0, 0]])
print("Para 3")
test([[0, 0, 0], [0, 0, 0], [0, 0, 0]])
print("Para [[0, 0]]")
test([[0, 0]])
print("Es 7")
test([[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]])
print("Es 11")
test([
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0], 
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0]
    ])


Para [[0]]
0
Para 2
3
Para 3
5
Para [[0, 0]]
2
Es 7
7
Es 11
21


In [11]:
# def reconstruct_path(came_from, current):
#     total_path = [current]
#     while current in came_from.keys():
#         current = came_from[current]
#         total_path.append(current)
#     return total_path

In [12]:
def is_useful_removal(location, map):
    """ Does making this cell passable create a path or just a dead end? """
    if map.passable[location[0]][location[1]]:
        return False
    neighbours = map.get_passable_neighbours(location)
    if len(neighbours) <= 1:
        return False
    return True

def useful_removals(map):
    useful_removals = filter(lambda loc: is_useful_removal(loc, map), map.locations())
    return useful_removals

In [16]:
def solution(map):
    m = Map(map)
    current_min_length = float("inf")
    for removed_cell in useful_removals(m):
        m.passable[removed_cell[0]][removed_cell[1]] = False
        current_length = m.A_star()
        if current_length < current_min_length:
            current_min_length = current_length
        m.passable[removed_cell[0]][removed_cell[1]] = True
    return current_min_length

In [22]:
print("Es 7")
print(solution([[0, 1, 1, 0], [0, 0, 0, 1], [1, 1, 0, 0], [1, 1, 1, 0]]))
print("Es 11")
print(solution([
        [0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 1, 0], 
        [0, 0, 0, 0, 0, 0],
        [0, 1, 1, 1, 1, 1],
        [0, 1, 1, 1, 1, 1],
        [0, 0, 0, 0, 0, 0]
    ])
)

Es 7
7
Es 11
11
